In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
folder_results = "results"
# folder_results = os.path.join("results", "test")
original_mcc_supervised, original_mcc_unsupervised = "MCC", "MCC_performance"
original_mcc_metrics = [original_mcc_supervised, original_mcc_unsupervised]
renamed_mcc_supervised, renamed_mcc_unsupervised = "MCC_supervised", "MCC_unsupervised"
renamed_mcc_metrics = [renamed_mcc_supervised, renamed_mcc_unsupervised]
renamed_avg_supervised, renamed_avg_unsupervised = renamed_mcc_supervised + "_avg", renamed_mcc_unsupervised + "_avg"
renamed_stability_supervised, renamed_stability_unsupervised = renamed_mcc_supervised + "_std", renamed_mcc_unsupervised + "_std"
labels_dict = {renamed_avg_supervised: "(supervised) MCC", renamed_avg_unsupervised: "(unsupervised) MCC", "algorithm": "Algorithm",
               "rank": "Rank", "missing_percentage": "% Incomplete samples", "dataset_type": "Dataset type", "amputation_mechanism": "Amputation Mechanism"}
unsupervised_metrics = ["silhouette", "db", "dunn"]
# metrics = supervised_metrics + unsupervised_metrics
indexes = ["dataset", "algorithm", "missing_percentage", "amputation_mechanism", "imputation", "run_n"]

## Multi-view algorithms

In [25]:
filelame = "incomplete_algorithms_evaluation.csv"
file_path = os.path.join(folder_results, filelame)
results = pd.read_csv(file_path)
print("results", results.shape)
results.head()

results (438000, 30)


/tmp/ipykernel_119700/3990048263.py:3: DtypeWarning: Columns (16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  results = pd.read_csv(file_path)


,dataset,algorithm,missing_percentage,amputation_mechanism,imputation,run_n,finished,completed,time_limited,n_samples,...,relative_cluster_sizes,y_true,y_pred,y_true_idx,y_pred_idx,silhouette,vrc,db,dbcv,dunn
0,simulated_gm,DAIMC,0,No,False,0,False,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,simulated_gm,DAIMC,0,No,False,1,False,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,simulated_gm,DAIMC,0,No,False,2,False,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,simulated_gm,DAIMC,0,No,False,3,False,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,simulated_gm,DAIMC,0,No,False,4,False,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
results = results[results["finished"]]
print("results", results.shape)

results (41121, 30)


In [27]:
results = results[results["completed"]]
print("results", results.shape)

results (41121, 30)


In [28]:
assert results["y_true_idx"].eq(results["y_pred_idx"]).all()
results["y_true_final"] = results.apply(lambda x: pd.Series(eval(x["y_true"]), index= eval(x["y_true_idx"])).to_list(), axis= 1)
results["y_pred_final"] = results.apply(lambda x: pd.Series(eval(x["y_pred"]), index= eval(x["y_pred_idx"])).to_list(), axis= 1)

In [6]:
results = results[results["completed"]]
mask = results["dataset"].str.startswith("simulated")
results["dataset_type"] = mask.where(mask, "real").mask(mask, "synthetic")
results = results.rename(columns = {original_mcc_supervised: renamed_mcc_supervised, original_mcc_unsupervised: renamed_mcc_unsupervised})
print("results", results.shape)

In [7]:
nomissing_results = results.copy()
nomissing_results = nomissing_results[nomissing_results["missing_percentage"] == 0]
nomissing_results = nomissing_results.groupby(["dataset", "algorithm", "imputation", "dataset_type"], sort=False)[renamed_mcc_metrics]
nomissing_results = pd.concat([nomissing_results.mean(), nomissing_results.std()], axis= 1)
nomissing_results.columns = (nomissing_results.columns[:2] + "_avg").to_list() + (nomissing_results.columns[2:] + "_std").to_list()
nomissing_results = nomissing_results.reset_index()
color_dict = {alg:col for alg,col in zip(nomissing_results["algorithm"].unique(), list(sns.color_palette(None, nomissing_results["algorithm"].nunique()).as_hex()))}
nomissing_results.loc[:,"color"] = nomissing_results["algorithm"].apply(lambda x: color_dict[x])
print("nomissing_results", nomissing_results.shape)
nomissing_results.head()

In [8]:
px.scatter(nomissing_results, x="algorithm", y= renamed_avg_supervised, error_y= renamed_stability_supervised, template= "simple_white", facet_col= "dataset",
           facet_col_wrap=3, width=1400, height=1000, color= "algorithm", labels= labels_dict)

In [7]:
px.scatter(nomissing_results, x="algorithm", y= renamed_avg_supervised, error_y= renamed_stability_supervised, template= "simple_white", facet_col= "dataset",
           facet_col_wrap=3, width=1400, height=1000, color= "algorithm", labels= labels_dict)

In [8]:
nomissing_results["rank"] = 0
for d in nomissing_results["dataset"]:
    d_nomissing_results = nomissing_results[nomissing_results["dataset"] == d].sort_values([renamed_avg_supervised, renamed_stability_supervised], ascending= [False, True])
    nomissing_results.loc[d_nomissing_results.index, "rank"] = d_nomissing_results[renamed_avg_supervised].rank()
rank = nomissing_results.groupby(["algorithm", "imputation", "dataset_type"], sort=False)["rank"].mean().to_list()
by_datasettype_nomissing_results = results.copy()
by_datasettype_nomissing_results = by_datasettype_nomissing_results[by_datasettype_nomissing_results["missing_percentage"] == 0]
by_datasettype_nomissing_results = by_datasettype_nomissing_results.groupby(["algorithm", "imputation", "dataset_type"], sort=False)[renamed_mcc_metrics]
by_datasettype_nomissing_results = pd.concat([by_datasettype_nomissing_results.mean(), by_datasettype_nomissing_results.std()], axis= 1)
by_datasettype_nomissing_results.columns = (by_datasettype_nomissing_results.columns[:2] + "_avg").to_list() + (by_datasettype_nomissing_results.columns[2:] + "_std").to_list()
by_datasettype_nomissing_results = by_datasettype_nomissing_results.reset_index()
color_dict = {alg:col for alg,col in zip(by_datasettype_nomissing_results["algorithm"].unique(), list(sns.color_palette(None, by_datasettype_nomissing_results["algorithm"].nunique()).as_hex()))}
by_datasettype_nomissing_results.loc[:,"color"] = by_datasettype_nomissing_results["algorithm"].apply(lambda x: color_dict[x])
by_datasettype_nomissing_results["rank"] = rank
by_datasettype_nomissing_results["size"] = 1
print("by_datasettype_nomissing_results", by_datasettype_nomissing_results.shape)
by_datasettype_nomissing_results.head()

In [9]:
px.scatter(by_datasettype_nomissing_results, x="algorithm", y= renamed_avg_supervised, error_y= renamed_stability_supervised,
           template= "simple_white", facet_col = "dataset_type", size= "rank", color= "algorithm", labels= labels_dict)

In [10]:
px.scatter(by_datasettype_nomissing_results, x="rank", y= renamed_avg_supervised, error_y= renamed_stability_supervised, template= "simple_white", size= "size",
           facet_col = "dataset_type", text="algorithm", color= "algorithm", labels= labels_dict)

In [7]:
missing_results = results.copy()
missing_results = missing_results[missing_results["missing_percentage"] > 0]
missing_results = missing_results.groupby(["algorithm", "missing_percentage", "amputation_mechanism", "imputation", "dataset_type"], sort=False)[renamed_mcc_metrics]
missing_results = pd.concat([missing_results.mean(), missing_results.std()], axis= 1)
missing_results.columns = (missing_results.columns[:2] + "_avg").to_list() + (missing_results.columns[2:] + "_std").to_list()
missing_results = missing_results.reset_index()
color_dict = {alg:col for alg,col in zip(missing_results["algorithm"].unique(), list(sns.color_palette(None, missing_results["algorithm"].nunique()).as_hex()))}
missing_results.loc[:,"color"] = missing_results["algorithm"].apply(lambda x: color_dict[x])
# missing_results[renamed_stability_supervised] /= 5
# missing_results[renamed_stability_unsupervised] /= 2
missing_results["size"] = 1
print("missing_results", missing_results.shape)
missing_results.head()

In [12]:
fig = px.scatter(missing_results, x= renamed_avg_supervised, y= renamed_avg_unsupervised, animation_frame="missing_percentage", animation_group="algorithm",
                 size= "size", text="algorithm", color= "imputation", error_x=  renamed_stability_supervised,
                 error_y= renamed_stability_unsupervised, facet_row = "amputation_mechanism", facet_col = "dataset_type",
                 range_x=[0,1], range_y=[0,1], width=1400, height=2000, title= "Clustering performance on incomplete multi-view datasets",
                 template= "simple_white", labels= labels_dict)
fig

In [13]:
fig.write_html("test.html")

In [14]:
px.scatter(missing_results[missing_results["amputation_mechanism"] == "EDM"], x= renamed_avg_supervised, 
                 y= renamed_avg_unsupervised, animation_frame="missing_percentage", animation_group="algorithm",
                 size= "size", text="algorithm", color= "imputation", error_x=  renamed_stability_supervised,
                 error_y= renamed_stability_unsupervised, facet_row = "amputation_mechanism", facet_col = "dataset_type",
                 range_x=[0,1], range_y=[0,1], height=600, title= "Clustering performance on incomplete multi-view datasets",
                 template= "simple_white", labels= labels_dict)

In [15]:
missing_results = results.copy()
missing_results = missing_results[missing_results["missing_percentage"] > 0]
missing_results = missing_results.groupby(["algorithm", "missing_percentage", "amputation_mechanism", "imputation", "dataset_type"], sort=False)[supervised_metrics]
missing_results = pd.concat([missing_results.mean(), missing_results.std()], axis= 1)
missing_results.columns = (missing_results.columns[:2] + "_avg").to_list() + (missing_results.columns[2:] + "_std").to_list()
missing_results = missing_results.reset_index()
missing_results["rank"] = 0
for d in missing_results["dataset"]:
    d_missing_results = missing_results[missing_results["dataset"] == d].sort_values(["MCC_avg", "MCC_std"], ascending= [False, True])
    missing_results.loc[d_missing_results.index, "rank"] = d_missing_results["MCC_avg"].rank()


missing_results = results.copy()
missing_results = missing_results[missing_results["missing_percentage"] > 0]
missing_results = missing_results.groupby(["algorithm", "missing_percentage", "amputation_mechanism", "imputation", "dataset_type"], sort=False)[supervised_metrics]
missing_results = pd.concat([missing_results.mean(), missing_results.std()], axis= 1)
missing_results.columns = (missing_results.columns[:2] + "_avg").to_list() + (missing_results.columns[2:] + "_std").to_list()
missing_results = missing_results.reset_index()
color_dict = {alg:col for alg,col in zip(missing_results["algorithm"].unique(), list(sns.color_palette(None, missing_results["algorithm"].nunique()).as_hex()))}
missing_results.loc[:,"color"] = missing_results["algorithm"].apply(lambda x: color_dict[x])
missing_results["MCC_std"] = missing_results["MCC_std"]/10
missing_results["MCC_performance_std"] = missing_results["MCC_performance_std"]/5
print("missing_results", missing_results.shape)
missing_results.head()